In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import torch
import itertools
import string
import numpy as np
from IPython.display import clear_output
from matplotlib import pyplot as plt
from gensim.models import word2vec
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

from sklearn import metrics
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/df_difficult_26_01.csv')
target_names = list(set(df['doc_name'].tolist()))
label2idx = {label: idx for idx, label in enumerate(target_names)}
print(label2idx)
df['doc_name_enc']=df['doc_name'].replace(label2idx)
df

{'Указ, региональный уровень, глава исполнительной власти': 0, 'Постановление, региональный уровень, законодательный орган': 1, 'Постановление, региональный уровень, органы исполнительной власти': 2, 'Постановление, региональный уровень, правительство': 3, 'Постановление, федеральный уровень, правительство': 4, 'Закон, региональный уровень, законодательный орган': 5, 'Приказ, региональный уровень, органы исполнительной власти': 6, 'Распоряжение, региональный уровень, глава исполнительной власти': 7, 'Распоряжение, федеральный уровень, правительство': 8, 'Указ, федеральный уровень, глава исполнительной власти': 9, 'Распоряжение, региональный уровень, правительство': 10, 'Закон, федеральный уровень, законодательный орган': 11, 'Распоряжение, федеральный уровень, глава исполнительной власти': 12, 'Постановление, региональный уровень, глава исполнительной власти': 13, 'Приказ, федеральный уровень, органы исполнительной власти': 14}


,doc_name,doc_text,text_token,text_token_filtred,text_difficult,doc_name_enc
0,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация ус...,распоряжение президент российский федерация ус...,комитет российский федерация установить числен...,12
1,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация пр...,распоряжение президент российский федерация пр...,российский федерация контроль оборот наркотик ...,12
2,"Указ, федеральный уровень, глава исполнительно...",указ президента российской федерации об освобо...,указ президент российский федерация освобожден...,указ президент российский федерация освобожден...,федерация контроль оборот наркотик освободить ...,9
3,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация по...,распоряжение президент российский федерация по...,многолетний плодотворный деятельность наградит...,12
4,"Распоряжение, федеральный уровень, глава испол...",распоряжение президента российской федерации о...,распоряжение президент российский федерация по...,распоряжение президент российский федерация по...,молодёжь объявить благодарность андропов игорь...,12
...,...,...,...,...,...,...
22886,"Указ, региональный уровень, глава исполнительн...",губернатор орловской области у к аз о мерах по...,губернатор орловский область мера проведение к...,губернатор орловский область мера проведение к...,проживать инвалид ветеран великий отечественны...,0
22887,"Указ, региональный уровень, глава исполнительн...",ммжхжчмитжкъщ а г х м башкортостан республи...,ммжхжчмитжкъщий башкортостан республикаь респу...,ммжхжчмитжкъщий башкортостан республикаь респу...,должность заместитель премьер министр правител...,0
22888,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края об отмене...,губернатор пермский край отмена ограничение ка...,губернатор пермский край отмена ограничение ка...,посёлок октябрьский соответствие закон российс...,0
22889,"Указ, региональный уровень, глава исполнительн...",у к а з губернатора пермского края г об утве...,губернатор пермский край утверждение порядок п...,губернатор пермский край утверждение порядок п...,должность пермский край отдельный должность го...,0


In [ ]:
import gensim.downloader as api
print("\n".join(api.info()['models'].keys()))

fasttext-wiki-news-subwords-300
conceptnet-numberbatch-17-06-300
word2vec-ruscorpora-300
word2vec-google-news-300
glove-wiki-gigaword-50
glove-wiki-gigaword-100
glove-wiki-gigaword-200
glove-wiki-gigaword-300
glove-twitter-25
glove-twitter-50
glove-twitter-100
glove-twitter-200
__testing_word2vec-matrix-synopsis


In [ ]:
word2vec_pre = api.load("glove-twitter-50")
tok = df['text_token_filtred'][:3].str.split().tolist()
word2vec_pre.most_similar(positive=['президент'])

[==================================================] 100.0% 199.5/199.5MB downloaded


[('премьер-министр', 0.864189863204956),
 ('путин', 0.8365647196769714),
 ('глава', 0.8298710584640503),
 ('заявил', 0.8257015943527222),
 ('мид', 0.8176457285881042),
 ('владимир', 0.8131812810897827),
 ('посол', 0.810960590839386),
 ('украины', 0.8079652190208435),
 ('янукович', 0.8058887720108032),
 ('грузии', 0.8036628365516663)]

Своя модель

In [ ]:
data_train, data_valid = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
texts = data_train['text_difficult'].tolist()
words = [i.split() for i in texts]
count_words = Counter()
for text in words:
  for i in text:
    count_words[i]+=1
more_10 = [k for k, v in count_words.items() if v>5]
words_more_10 = []
for t in words:
  passed = [i for i in t if i in more_10]
  words_more_10.append(passed)
X = np.array(words_more_10)
model_wv_dif = word2vec.Word2Vec(X, vector_size=50, window=5, min_count=1, workers=4)
model_wv_dif.wv.most_similar(positive=['президент'])

<ipython-input-6-cf65957199d8>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(words_more_10)


[('глава', 0.4891655147075653),
 ('правительство', 0.471372127532959),
 ('рref', 0.4617333710193634),
 ('россия', 0.44999435544013977),
 ('рпref', 0.4443207383155823),
 ('федерация', 0.43119654059410095),
 ('август', 0.42723774909973145),
 ('ndот', 0.42563751339912415),
 ('российский', 0.42250001430511475),
 ('казачество', 0.39450785517692566)]

In [ ]:

tfidf_dif  = TfidfVectorizer()
tfidf_dif.fit([' '.join(i) for i in data_train['text_difficult'].str.split()])
tf_dict_dif=dict(zip(tfidf_dif.get_feature_names_out(), tfidf_dif.idf_))

In [ ]:
def average_emb(w2v, text):

    doc = [word for word in text.split() if word in w2v.wv]

    return np.mean(w2v.wv[doc], axis=0)

def average_emb_tdidf(w2v, td_dict, text):

    doc = [word for word in text.split() if word in w2v.wv]
    tf = np.array([td_dict[word] for word in text.split() if word in w2v.wv])
    #print(w2v.wv[doc].dot(tf[0]))
    return np.mean(w2v.wv[doc].dot(tf[0]), axis=0)

def average_emb_pre(w2v, text):

    doc = [word for word in text.split() if word in w2v]

    return np.mean(w2v[doc], axis=0)

def average_emb_tdidf_pre(w2v, td_dict, text):

    doc = [word for word in text.split() if word in w2v]
    tf = np.array([td_dict[word] if word in td_dict.keys() else 0 for word in doc ])

    return np.mean(w2v[doc].dot(tf[0]), axis=0)

In [ ]:
df_w2v = df[['doc_name', 'text_difficult']].copy()

df_w2v['w2v_dif'] = df_w2v.apply(lambda x: average_emb(model_wv_dif, x['text_difficult']),axis=1)
df_w2v['w2v_dif_idf'] = df_w2v.apply(lambda x: average_emb_tdidf(model_wv_dif,tf_dict_dif, x['text_difficult']),axis=1)

df_w2v['w2v_dif_pre'] = df_w2v.apply(lambda x: average_emb_pre(word2vec_pre, x['text_difficult']),axis=1)
df_w2v['w2v_dif_idf_pre'] = df_w2v.apply(lambda x: average_emb_tdidf_pre(word2vec_pre,tf_dict_dif, x['text_difficult']),axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.71 {'lr_clf__class_weight': 'balanced', 'lr_clf__penalty': 'l2', 'lr_clf__solver': 'newton-cg'}


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='newton-cg',penalty='l2', class_weight='balanced', random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/optimize.py:203: UserWarning: Line Search failed
  warnings.warn("Line Search failed")


Pipeline(steps=[('lr_clf',
                 LogisticRegression(class_weight='balanced', random_state=42,
                                    solver='newton-cg'))])

In [ ]:
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.85      0.88      0.87       329
                Закон, федеральный уровень, законодательный орган       0.92      0.93      0.93       347
 Постановление, региональный уровень, глава исполнительной власти       0.66      0.62      0.64       318
       Постановление, региональный уровень, законодательный орган       0.92      0.90      0.91       302
Постановление, региональный уровень, органы исполнительной власти       0.91      0.92      0.91       244
               Постановление, региональный уровень, правительство       0.64      0.62      0.63       318
                Постановление, федеральный уровень, правительство       0.79      0.84      0.81       314
       Приказ, региональный уровень, органы исполнительной власти       0.85      0.87      0.86       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Best score and parameter combination:
0.646 {'sgd_clf__class_weight': 'balanced', 'sgd_clf__loss': 'log', 'sgd_clf__penalty': 'l2'}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='log_loss',penalty='l2', class_weight='balanced', random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.87      0.84      0.86       329
                Закон, федеральный уровень, законодательный орган       0.88      0.95      0.91       347
 Постановление, региональный уровень, глава исполнительной власти       0.61      0.62      0.61       318
       Постановление, региональный уровень, законодательный орган       0.89      0.92      0.90       302
Постановление, региональный уровень, органы исполнительной власти       0.93      0.89      0.91       244
               Постановление, региональный уровень, правительство       0.58      0.58      0.58       318
                Постановление, федеральный уровень, правительство       0.81      0.76      0.78       314
       Приказ, региональный уровень, органы исполнительной власти       0.86      0.80      0.83       325
        Приказ, федеральный уровень,

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.6804999999999999 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.78      0.89      0.83       329
                Закон, федеральный уровень, законодательный орган       0.91      0.91      0.91       347
 Постановление, региональный уровень, глава исполнительной власти       0.63      0.59      0.61       318
       Постановление, региональный уровень, законодательный орган       0.94      0.84      0.89       302
Постановление, региональный уровень, органы исполнительной власти       0.97      0.87      0.92       244
               Постановление, региональный уровень, правительство       0.53      0.65      0.58       318
                Постановление, федеральный уровень, правительство       0.77      0.83      0.80       314
       Приказ, региональный уровень, органы исполнительной власти       0.81      0.80      0.81       325
        Приказ, федеральный уровень,

Вместе с td-idf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif_idf'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.6995 {'lr_clf__class_weight': None, 'lr_clf__penalty': 'l2', 'lr_clf__solver': 'sag'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='sag',penalty='l2', class_weight=None, random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.85      0.88      0.86       329
                Закон, федеральный уровень, законодательный орган       0.92      0.94      0.93       347
 Постановление, региональный уровень, глава исполнительной власти       0.63      0.63      0.63       318
       Постановление, региональный уровень, законодательный орган       0.90      0.90      0.90       302
Постановление, региональный уровень, органы исполнительной власти       0.92      0.90      0.91       244
               Постановление, региональный уровень, правительство       0.65      0.64      0.65       318
                Постановление, федеральный уровень, правительство       0.77      0.85      0.81       314
       Приказ, региональный уровень, органы исполнительной власти       0.86      0.87      0.86       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.6074999999999999 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'hinge', 'sgd_clf__penalty': None}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='hinge',penalty=None, class_weight=None, random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.78      0.78      0.78       329
                Закон, федеральный уровень, законодательный орган       0.89      0.84      0.86       347
 Постановление, региональный уровень, глава исполнительной власти       0.41      0.28      0.33       318
       Постановление, региональный уровень, законодательный орган       0.88      0.85      0.87       302
Постановление, региональный уровень, органы исполнительной власти       0.84      0.86      0.85       244
               Постановление, региональный уровень, правительство       0.39      0.57      0.46       318
                Постановление, федеральный уровень, правительство       0.76      0.69      0.73       314
       Приказ, региональный уровень, органы исполнительной власти       0.90      0.61      0.72       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.6705 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.79      0.88      0.83       329
                Закон, федеральный уровень, законодательный орган       0.89      0.90      0.90       347
 Постановление, региональный уровень, глава исполнительной власти       0.60      0.63      0.62       318
       Постановление, региональный уровень, законодательный орган       0.94      0.85      0.89       302
Постановление, региональный уровень, органы исполнительной власти       0.95      0.86      0.90       244
               Постановление, региональный уровень, правительство       0.53      0.65      0.59       318
                Постановление, федеральный уровень, правительство       0.70      0.84      0.76       314
       Приказ, региональный уровень, органы исполнительной власти       0.81      0.79      0.80       325
        Приказ, федеральный уровень,

Преобученный

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif_pre'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.6034999999999999 {'lr_clf__class_weight': None, 'lr_clf__penalty': None, 'lr_clf__solver': 'saga'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='saga',penalty=None, class_weight=None, random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.73      0.80      0.77       329
                Закон, федеральный уровень, законодательный орган       0.85      0.84      0.84       347
 Постановление, региональный уровень, глава исполнительной власти       0.52      0.47      0.50       318
       Постановление, региональный уровень, законодательный орган       0.76      0.76      0.76       302
Постановление, региональный уровень, органы исполнительной власти       0.85      0.85      0.85       244
               Постановление, региональный уровень, правительство       0.52      0.52      0.52       318
                Постановление, федеральный уровень, правительство       0.66      0.72      0.68       314
       Приказ, региональный уровень, органы исполнительной власти       0.65      0.70      0.67       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Best score and parameter combination:
0.5485 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'log', 'sgd_clf__penalty': 'l1'}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='log_loss',penalty='l1', class_weight=None, random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.68      0.80      0.74       329
                Закон, федеральный уровень, законодательный орган       0.82      0.86      0.84       347
 Постановление, региональный уровень, глава исполнительной власти       0.49      0.14      0.21       318
       Постановление, региональный уровень, законодательный орган       0.73      0.76      0.75       302
Постановление, региональный уровень, органы исполнительной власти       0.88      0.84      0.86       244
               Постановление, региональный уровень, правительство       0.60      0.31      0.40       318
                Постановление, федеральный уровень, правительство       0.68      0.68      0.68       314
       Приказ, региональный уровень, органы исполнительной власти       0.59      0.68      0.63       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.6 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=700,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.65      0.84      0.73       329
                Закон, федеральный уровень, законодательный орган       0.82      0.86      0.84       347
 Постановление, региональный уровень, глава исполнительной власти       0.65      0.51      0.57       318
       Постановление, региональный уровень, законодательный орган       0.85      0.76      0.80       302
Постановление, региональный уровень, органы исполнительной власти       0.94      0.86      0.90       244
               Постановление, региональный уровень, правительство       0.48      0.58      0.52       318
                Постановление, федеральный уровень, правительство       0.65      0.80      0.71       314
       Приказ, региональный уровень, органы исполнительной власти       0.69      0.72      0.70       325
        Приказ, федеральный уровень,

Предобученный с td-idf

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_w2v['w2v_dif_idf_pre'], df_w2v['doc_name'], test_size=0.2, random_state=42)
X_train = np.vstack(X_train)
X_test = np.vstack(X_test)

In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(random_state=42))
])

parameters = {
              'lr_clf__solver':['lbfgs', 'newton-cg', 'sag', 'saga'],
              'lr_clf__class_weight':[None, 'balanced'],
              'lr_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],
              }

model = GridSearchCV(lr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
56 fits failed out of a total of 128.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
8 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
 

Best score and parameter combination:
0.58 {'lr_clf__class_weight': None, 'lr_clf__penalty': None, 'lr_clf__solver': 'sag'}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
lr_ppl_clf = Pipeline([

    ('lr_clf', LogisticRegression(solver='sag',penalty=None, class_weight=None, random_state=42))
])
lr_ppl_clf.fit(X_train, y_train)
predicted_lr = lr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_lr))
print(metrics.accuracy_score(y_test, predicted_lr))
print(metrics.precision_score(y_test, predicted_lr, average='macro'))
print(metrics.recall_score(y_test, predicted_lr, average='macro'))
print(metrics.f1_score(y_test, predicted_lr, average='macro'))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.70      0.79      0.74       329
                Закон, федеральный уровень, законодательный орган       0.82      0.84      0.83       347
 Постановление, региональный уровень, глава исполнительной власти       0.53      0.43      0.48       318
       Постановление, региональный уровень, законодательный орган       0.73      0.75      0.74       302
Постановление, региональный уровень, органы исполнительной власти       0.83      0.86      0.84       244
               Постановление, региональный уровень, правительство       0.50      0.56      0.53       318
                Постановление, федеральный уровень, правительство       0.63      0.72      0.67       314
       Приказ, региональный уровень, органы исполнительной власти       0.65      0.68      0.67       325
        Приказ, федеральный уровень,

In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(random_state=42))
])

parameters = {
              'sgd_clf__loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd_clf__class_weight':[None, 'balanced'],
              'sgd_clf__penalty':[None, 'l2', 'l1', 'elasticnet'],

              }
model = GridSearchCV(sgd_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.52 {'sgd_clf__class_weight': None, 'sgd_clf__loss': 'perceptron', 'sgd_clf__penalty': None}


In [ ]:
sgd_ppl_clf = Pipeline([

    ('sgd_clf', SGDClassifier(loss='perceptron',penalty=None, class_weight=None, random_state=42))
])
sgd_ppl_clf.fit(X_train, y_train)


predicted_sgd_val = sgd_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, predicted_sgd_val))
print(metrics.accuracy_score(y_test, predicted_sgd_val), 'accuracy')
print(metrics.precision_score(y_test, predicted_sgd_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, predicted_sgd_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, predicted_sgd_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.72      0.65      0.68       329
                Закон, федеральный уровень, законодательный орган       0.77      0.78      0.77       347
 Постановление, региональный уровень, глава исполнительной власти       0.57      0.03      0.05       318
       Постановление, региональный уровень, законодательный орган       0.63      0.75      0.68       302
Постановление, региональный уровень, органы исполнительной власти       0.74      0.80      0.77       244
               Постановление, региональный уровень, правительство       0.63      0.14      0.23       318
                Постановление, федеральный уровень, правительство       0.52      0.67      0.59       314
       Приказ, региональный уровень, органы исполнительной власти       0.31      0.81      0.45       325
        Приказ, федеральный уровень,

In [ ]:
fr_ppl_clf = Pipeline([

    ('rf_clf', RandomForestClassifier(random_state=42))
])

parameters = {
              'rf_clf__n_estimators':[300, 500, 700],
              'rf_clf__max_depth':[3, 5, 8, 11],


              }
model = GridSearchCV(fr_ppl_clf, parameters, cv=4, n_jobs=-1).fit(X_train[:2000], y_train[:2000])
print('Best score and parameter combination:')
print(model.best_score_, model.best_params_)

Best score and parameter combination:
0.5445 {'rf_clf__max_depth': 11, 'rf_clf__n_estimators': 700}


In [ ]:
fr_ppl_clf = Pipeline([

    ('sgd_clf', RandomForestClassifier(n_estimators=300,max_depth=11,  random_state=42))
])
fr_ppl_clf.fit(X_train, y_train)


ppredicted_fr_val = fr_ppl_clf.predict(X_test)
print(metrics.classification_report(y_test, ppredicted_fr_val))
print(metrics.accuracy_score(y_test, ppredicted_fr_val), 'accuracy')
print(metrics.precision_score(y_test, ppredicted_fr_val, average='macro'), 'precision')
print(metrics.recall_score(y_test, ppredicted_fr_val, average='macro'), 'recall')
print(metrics.f1_score(y_test, ppredicted_fr_val, average='macro'), 'f1')

                                                                   precision    recall  f1-score   support

               Закон, региональный уровень, законодательный орган       0.59      0.76      0.66       329
                Закон, федеральный уровень, законодательный орган       0.74      0.87      0.80       347
 Постановление, региональный уровень, глава исполнительной власти       0.56      0.47      0.51       318
       Постановление, региональный уровень, законодательный орган       0.81      0.74      0.77       302
Постановление, региональный уровень, органы исполнительной власти       0.93      0.84      0.88       244
               Постановление, региональный уровень, правительство       0.42      0.49      0.45       318
                Постановление, федеральный уровень, правительство       0.64      0.74      0.68       314
       Приказ, региональный уровень, органы исполнительной власти       0.61      0.67      0.64       325
        Приказ, федеральный уровень,